<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/texygen_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recursive https://github.com/JuanJoseMV/neuraltextgen.git
!pip install -r /content/neuraltextgen/texygen/requirements.txt
!pip install simpletransformers

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'neuraltextgen'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 36 (delta 16), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (36/36), done.
Submodule 'texygen' (https://github.com/geek-ai/Texygen.git) registered for path 'texygen'
Cloning into '/content/neuraltextgen/texygen'...
remote: Enumerating objects: 888, done.        
remote: Total 888 (delta 0), reused 0 (delta 0), pack-reused 888        
Receiving objects: 100% (888/888), 21.85 MiB | 5.02 MiB/s, done.
Resolving deltas: 100% (537/537), done.
Submodule path 'texygen': checked out '3104e22ac75f3cc2070da2bf5e2da6d2bef149ad'
     |████████████████████████████████| 225kB 7.1MB/s 
     |████████████████████████████████| 1.2MB 13.6MB/s 
     |████████████████████████████████| 8.2MB 32.9MB/s 
     |████████████████████████████████| 1.8MB 44.2MB/s 
     |████████████████████████████████| 122kB 55.4MB/s 
     |

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.11 MiB | 10.04 MiB/s, done.
Resolving deltas: 100% (5464/5464), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-mazmop3p
Created temporary directory: /tmp/pip-req-tracker-2qu5kq8d
Created requirements tracker '/tmp/pip-req-tracker-2qu5kq8d'
Created temporary directory: /tmp/pip-install-2b2elkh6
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-yw5lsn65
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-2qu5kq8d'
    Running setup.py (

In [2]:
import nltk
nltk.download('punkt')

import sys, os
import os
os.chdir("/content/neuraltextgen/texygen")
from utils.metrics.Bleu import Bleu
from utils.metrics.SelfBleu import SelfBleu

os.chdir("/content/neuraltextgen/")
from NeuralTextGenerator import BertTextGenerator

os.chdir("/content")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
bert_model = BertTextGenerator('bert-base-uncased')

parameters = {'n_samples': 100,  # 1000
              'batch_size': 50,  # 50
              'max_len': 40,
              'top_k': 100,
              'temperature': 1,
              'burnin': 250,
              'sample': True,
              'max_iter': 500,
              'seed_text': ""
              }

# "key1=val1_key2=val2_...txt"
file_path = "_".join([f"{k}={v}" for k, v in parameters.items()])+".txt"

bert_sents = bert_model.generate(save_to_path=file_path, **parameters)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Finished batch 1 in 72.484s
Finished batch 2 in 76.944s


In [5]:
print("\nEnglish text generated: ")
for sent in bert_sents:
  print(f"\t{sent}")


English text generated: 
	 and getting started seemed to make jade the crazy, crazy, crazy teenager she was. maybe she had already gotten started really well, but it had all been done all too well in her own mind. 
	 if i'm ill, i cannot walk. if it becomes too hard to walk, death will follow, but should i use my bow and stay here for one or two nights after the funeral? 
	 consisting of mad culture members carlos huerta, gary merrill, and emily dickinson ( dickinson and the latter longtime cast members ), no doubt cancelled its nine - week run, with mad culture absent again. 
	 farewell, farewell, farewell, farewell to the faerie, and once again to us, and our father. farewell, to the shore of the lake, the tree, the glass lake, the tree. 
	 cover : " me and you " in 1985, during a tour and journey through south america, the group visited several countries and was questioned ( particularly with regard to the country's current status ). 
	 hal epplery, the announcer on each original p

#Evaluation - Original

In [6]:
from nltk.translate import bleu_score as bleu

def prepare_data(data_file, replacements={}, uncased=True):
    data = [d.strip().split() for d in open(data_file, 'r').readlines()]
    if uncased:
        data = [[t.lower() for t in sent] for sent in data]
        
    for k, v in replacements.items():
        data = [[t if t != k else v for t in sent] for sent in data]
 
    return data

def prepare_wiki(data_file, uncased=True):
    replacements = {"@@unknown@@": "[UNK]"}
    return prepare_data(data_file, replacements=replacements, uncased=uncased)

def prepare_tbc(data_file):        
    replacements = {"``": "\"", "\'\'": "\""}
    return prepare_data(data_file, replacements=replacements)

def corpus_bleu(generated, references):
    """ Compute similarity between two corpora as measured by
    comparing each sentence of `generated` against all sentences in `references` 
    
    args:
        - generated (List[List[str]]): list of sentences (split into tokens)
        - references (List[List[str]]): list of sentences (split into tokens)
        
    returns:
        - bleu (float)
    """    
    return bleu.corpus_bleu([references for _ in range(len(generated))], generated)

In [7]:
!git clone https://github.com/nyu-dl/bert-gen
wiki103_file = 'bert-gen/data/wiki103.5k.txt'
tbc_file = 'bert-gen/data/tbc.5k.txt'

wiki_data = prepare_wiki(wiki103_file)
tbc_data = prepare_tbc(tbc_file)

Cloning into 'bert-gen'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


Try to evaluate using original functions and no cleaning of wiki-data

In [8]:
print("BERT-TBC BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data)))
print("BERT-Wiki103 BLEU: %.2f" % (100 * corpus_bleu(bert_sents, wiki_data)))
print("BERT-{TBC + Wiki103} BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data[:2500] + wiki_data[:2500])))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BERT-TBC BLEU: 17.31
BERT-Wiki103 BLEU: 23.39
BERT-{TBC + Wiki103} BLEU: 22.46


Try to evaluate after cleaning

In [9]:
def cleaner(data):
  len_mask = []
  for i in range(len(data)):
    if len(data[i]) <4:
      len_mask.append(False)
    else:
      len_mask.append(True)

  data = [b for a, b in zip(len_mask, data) if a]
  return data

wiki_data = cleaner(wiki_data)
tbc_data = cleaner(tbc_data)

print("BERT-TBC BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data)))
print("BERT-Wiki103 BLEU: %.2f" % (100 * corpus_bleu(bert_sents, wiki_data)))
print("BERT-{TBC + Wiki103} BLEU: %.2f" % (100 * corpus_bleu(bert_sents, tbc_data[:2500] + wiki_data[:2500])))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BERT-TBC BLEU: 17.31
BERT-Wiki103 BLEU: 23.39
BERT-{TBC + Wiki103} BLEU: 22.46


## Evaluation - Texygen

In [15]:
bleu_score_tbc = Bleu(file_path, tbc_file)
bleu_score_wiki = Bleu(file_path, wiki103_file)

print("(Texygen) BERT-TBC BLEU: %.2f" % (100 * bleu_score_tbc.get_bleu()))
print("(Texygen) BERT-Wiki103 BLEU: %.2f" % (100 * bleu_score_wiki.get_bleu()))



(Texygen) BERT-TBC BLEU: 17.43
(Texygen) BERT-Wiki103 BLEU: 18.25
